In [7]:
import numpy as np
from imutils import contours
import cv2
import os

root_directory = r"C:\Users\User\Desktop\New Votes DataSet\Dataset_8"
for subdir, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(subdir, file)
            image = cv2.imread(image_path)
            dim=(1398,1783)
            image=cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


            cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area <50:
                    cv2.drawContours(thresh, [c], -1, 0, -1)


            invert = 255 - thresh
            offset, old_cY, old_cX, first = 10, 0, 0, True
            visualize = cv2.cvtColor(invert, cv2.COLOR_GRAY2BGR)
            
            cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
            crosscount=-1
            table = []
            row = []
            prevX=0
            current_row_index = -1 
            current_column_index = -1
            column_centroids = []
            value= 0
            value1=0
            approxval=None
            processed_centroids = []  
            outx=0
            outy=0
            size=0
            party= None
            terminate_main_loop= False
            for c in cnts:
                M = cv2.moments(c)
                if M["m00"] > 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])

                    if 0 <= cX <= 1000 or 0<=cY<=100:
                        continue

                    too_close = False
                    for (prev_cX, prev_cY) in processed_centroids:
                        if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200: 
                            too_close = True
                            break

                    if too_close:
                        continue  


                    processed_centroids.append((cX, cY))

                    if abs(cY - old_cY) > offset:
                        if not first:
                            table.append(row)
                        old_cY = cY
                        row = []
                        current_row_index += 1


                    column_index = -1
                    for i, column_cX in enumerate(column_centroids):
                        if abs(cX - column_cX) < 50:
                            column_index = i
                            break

                    if column_index == -1:
                        column_index = len(column_centroids)
                        column_centroids.append(cX)

                    if column_index ==0:
                        
                        x, y, w, h= cv2.boundingRect(c)
                        
                        roi =invert[y:y + h, x:x + w]
                        crosscount=crosscount+1
                        roi_cnts = cv2.findContours(roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        roi_cnts = roi_cnts[0] if len(roi_cnts) == 2 else roi_cnts[1]

                        for roi_c in roi_cnts:
                                
                                epsilon = 0.04 * cv2.arcLength(roi_c, True)
                                approx = cv2.approxPolyDP(roi_c, epsilon, True)
                                num_vertices = len(approx)
                                if(num_vertices>4):
                                    
                                    if value<2:
                                        print(f'num vertices {num_vertices} cx{cX} prevx {prevX}')
                                        if(num_vertices==8):
                                            
                                            if prevX==0:
                                                
                                                value=value+1
                                                prevX=cX
                                                value1=crosscount
                                                approxval=approx
                                                size=len(approx)
                                                outx=x
                                                outy=y
                                            else:
                                                if prevX==cX:
                                                    value=value-1
                                                    value1=crosscount
                                                    approxval=approx
                                                    size=len(approx)
                                                    outx=x
                                                    outy=y
                                                    value=value+1
                                                else:
                                                    value=value+1

                                        else:
                                            if prevX!=cX:
                                                value=value+1
                                                prevX=cX
                                                
                                            
                                            
                                    else:
                                        
                                        terminate_main_loop=True
                                        break
                                         
                    if terminate_main_loop:
                        break
            
            if value==1:
                x2,y2, w2, h2 = cv2.boundingRect(approxval)
                x2 += outx
                y2 += outy
                cv2.rectangle(visualize, (x2, y2), (x2 +w2, y2 + h2), (0, 0, 255)), 
                cv2.putText(
                                                visualize,
                                                f"GG: ({value1}, {0})",
                                                (x2-20,y2-20),
                                                cv2.FONT_HERSHEY_SIMPLEX,
                                                0.8,
                                                (0,0,255),
                                                2,
                                            )
                
                
                print(f'X value :{outx} Number of crosses:{value} Row index {value1} size{size}')
                if size==8:
                    
                    if value1==0:
                        party="PartyA"
                    elif value1==1:
                        party="PartyB"
                    elif value1==2:
                        party="PartyC"
                    elif value1==3:
                        party="PartyD"
                    elif value1==4:
                        party="PartyE"
                    else:
                        party="PartyF"
                    
            print(party)
            scale_percent = 40 
            width = int(visualize.shape[1] * scale_percent / 100)
            height = int(visualize.shape[0] * scale_percent / 100)
            dim = (width, height)




            resized_visualize = cv2.resize(visualize, dim, interpolation=cv2.INTER_AREA)
            height, width, channels =resized_visualize.shape
            cv2.imshow('Visualize', resized_visualize)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


num vertices 5 cx1198 prevx 0
num vertices 5 cx1198 prevx 1198
num vertices 6 cx1198 prevx 1198
X value :0 Number of crosses:1 Row index 0 size0
None
num vertices 6 cx1178 prevx 0
num vertices 5 cx1178 prevx 1178
num vertices 6 cx1178 prevx 1178
num vertices 6 cx1178 prevx 1178
num vertices 6 cx1179 prevx 1178
None
num vertices 6 cx1136 prevx 0
X value :0 Number of crosses:1 Row index 0 size0
None
num vertices 8 cx1213 prevx 0
num vertices 7 cx1213 prevx 1213
num vertices 6 cx1213 prevx 1213
num vertices 6 cx1211 prevx 1213
None
num vertices 8 cx1207 prevx 0
X value :1088 Number of crosses:1 Row index 0 size8
PartyA
num vertices 8 cx1210 prevx 0
X value :1091 Number of crosses:1 Row index 0 size8
PartyA
num vertices 8 cx1194 prevx 0
X value :1076 Number of crosses:1 Row index 0 size8
PartyA
num vertices 8 cx1211 prevx 0
X value :1093 Number of crosses:1 Row index 0 size8
PartyA
num vertices 8 cx1211 prevx 0
X value :1092 Number of crosses:1 Row index 0 size8
PartyA
num vertices 8 cx120

In [6]:
import numpy as np
from imutils import contours
import cv2


image = cv2.imread(r"C:\Users\User\Desktop\New Votes DataSet\Dataset_8\Party 06\1733843622551.jpg")
dim=(1398,1783)
image=cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area <10:
        cv2.drawContours(thresh, [c], -1, 0, -1)


invert = 255 - thresh
offset, old_cY, old_cX, first = 10, 0, 0, True
visualize = cv2.cvtColor(invert, cv2.COLOR_GRAY2BGR)


cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
crosscount=0
table = []
row = []
rownumber=0
current_row_index = -1 
current_column_index = -1
count=0
excluded_row_y_range = (600,900)
column_centroids = []
values = []
x_marks = []
processed_centroids = []  
black_pixel_counter = 0
for c in cnts:
    M = cv2.moments(c)
    if M["m00"] > 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        if 0 <= cX <= 1000:
            continue
            
        too_close = False
        for (prev_cX, prev_cY) in processed_centroids:
            if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200: 
                too_close = True
                break

        if too_close:
            continue  


        processed_centroids.append((cX, cY))


        if abs(cY - old_cY) > offset:
            if not first:
                table.append(row)
            old_cY = cY
            row = []
            current_row_index += 1

        column_index = -1
        for i, column_cX in enumerate(column_centroids):
            if abs(cX - column_cX) < 50:
                column_index = i
                break

        if column_index == -1:
            column_index = len(column_centroids)
            column_centroids.append(cX)
            
        if column_index ==0:
            for inner_c in cnts:
#                 if crosscount>2:
#                     break
                inner_M = cv2.moments(inner_c)
                if inner_M["m00"] > 0:
                    inner_cX = int(inner_M["m10"] / inner_M["m00"])
                    inner_cY = int(inner_M["m01"] / inner_M["m00"])
                    if abs(inner_cX - cX) < 50:  
                        inner_epsilon = 0.04 * cv2.arcLength(inner_c, True)
                        inner_approx = cv2.approxPolyDP(inner_c, inner_epsilon, True)
#                         print(len(inner_approx) )
                        if len(inner_approx)>4:
                            crosscount=crosscount+1
                            x, y, w, h = cv2.boundingRect(inner_approx)
                            roi =visualize[y:y+h, x:x+w]
                            if np.any(roi == 0):
                                black_pixel_counter=black_pixel_counter+1
                                cv2.rectangle(visualize, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            

            for inner_c in cnts:
                if count>=1:
                    break
                inner_M = cv2.moments(inner_c)
                if inner_M["m00"] > 0:
                    inner_cX = int(inner_M["m10"] / inner_M["m00"])
                    inner_cY = int(inner_M["m01"] / inner_M["m00"])
                    if abs(inner_cX - cX) < 50:  
                        inner_epsilon = 0.04 * cv2.arcLength(inner_c, True)
                        inner_approx = cv2.approxPolyDP(inner_c, inner_epsilon, True)
                        rownumber=rownumber+1
#                         print(len(inner_approx))
                        if len(inner_approx) == 8:
                            count=count+1
                            ix, iy, iw, ih = cv2.boundingRect(inner_approx)
                            cv2.rectangle(visualize, (ix, iy), (ix + iw, iy + ih), (255, 0, 0), 2)
                            cv2.putText(
                                visualize,
                                f"Detected: ({rownumber-2}, {column_index})",
                                (inner_cX - 50, inner_cY - 10),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.8,
                                (0, 255,0),
                                2,
                            )

print(black_pixel_counter)
scale_percent = 40 
width = int(visualize.shape[1] * scale_percent / 100)
height = int(visualize.shape[0] * scale_percent / 100)
dim = (width, height)


resized_visualize = cv2.resize(visualize, dim, interpolation=cv2.INTER_AREA)


cv2.imshow('Visualize', resized_visualize)
cv2.waitKey(0)
cv2.destroyAllWindows() 

1


In [3]:
import os
import random
import shutil

# Path to the original dataset directory
original_dataset_path = r"C:\Users\User\Desktop\New Votes DataSet\New Votes DataSet"

# Output directory for the generated datasets
output_base_path = r"C:\Users\User\Desktop\New Votes DataSet"
os.makedirs(output_base_path, exist_ok=True)

# Number of datasets to generate
num_datasets = 17

# Create 17 datasets
for i in range(1, num_datasets + 1):
    dataset_path = os.path.join(output_base_path, f'Dataset_{i}')
    os.makedirs(dataset_path, exist_ok=True)

    # Process each class
    for cls in os.listdir(original_dataset_path):
        class_path = os.path.join(original_dataset_path, cls)
        if os.path.isdir(class_path):
            # Get all images in the current class
            images = os.listdir(class_path)

            # Determine the number of images for the current dataset
            num_images = random.randint(1, len(images))

            # Randomly select the images
            selected_images = random.sample(images, num_images)

            # Create class folder in the new dataset
            new_class_path = os.path.join(dataset_path, cls)
            os.makedirs(new_class_path, exist_ok=True)

            # Copy the selected images to the new dataset
            for image in selected_images:
                src = os.path.join(class_path, image)
                dst = os.path.join(new_class_path, image)
                shutil.copy(src, dst)

print(f"Generated {num_datasets} datasets at {output_base_path}")


Generated 17 datasets at C:\Users\User\Desktop\New Votes DataSet


In [ ]:
pip install imutils

In [ ]:
pip install pytesseract